# Genetic algorithms
They are called upon when traditional algorithmic approaches are insufficient for arriving at a solution to a problem in a reasonable amount of time. In other words, genetic algorithms are usually reserved for complex problems without easy solutions. 

A genetic algorithm includes a population (group) of individuals known as chromosomes. The chromosomes, each composed of genes that specify their traits, are competing to solve some problem. How well a chromosome solves a problem is defined by a fitness function.

The genetic algorithm goes through generations. In each generation, the chromosomes that are more fit are more likely to be selected to reproduce. There is also a probability in each generation that two chromosomes will have their genes merged. This is known as crossover. And finally, there is the important possibility in each generation that a gene in a chromosome may mutate (randomly change).

After the fitness function of some individual in the population crosses some specified threshold, or the algorithm runs through some specified maximum number of generations, the best individual (the one that scored highest in the fitness function) is returned.

Genetic algorithms are not a good solution for all problems. They depend on three partially or fully stochastic (randomly determined) operations: selection, crossover, and mutation. Therefore, they may not find an optimal solution in a reasonable amount of time. For most problems, more deterministic algorithms exist with better guarantees. But there are problems for which no fast deterministic algorithm exists. In these cases, genetic algorithms are a good choice. 

We will start by defining an interface for the individuals that the generic algorithm can operate on. The abstract class Chromosome defines four essential features. A chromosome must be able to do the following:

- Determine its own fitness
- Create an instance with randomly selected genes (for use in filling the first generation)
- Implement crossover (combine itself with another of the same type to create children)—in other words, mix itself with another chromosome
- Mutate—make a small, fairly random change in itself

In [1]:
from __future__ import annotations
from typing import TypeVar, Tuple, Type
from abc import ABC, abstractmethod

T = TypeVar('T', bound='Chromosome') # for returning self

# Base class for all chromosomes; all methods must be overridden
class Chromosome(ABC):
    @abstractmethod
    def fitness(self) -> float:
        ...

    @classmethod
    @abstractmethod
    def random_instance(cls: Type[T]) -> T:
        ...

    @abstractmethod
    def crossover(self: T, other: T) -> Tuple[T, T]:
        ...

    @abstractmethod
    def mutate(self) -> None:
        ...

We will implement the algorithm itself (the code that will manipulate chromosomes) as a generic class that is open to subclassing for future specialized applications.

GeneticAlgorithm takes a generic type that conforms to Chromosome, and its name is C. The enum SelectionType is an internal type used for specifying the selection method used by the algorithm. The two most common genetic algorithm selection methods are known as roulette-wheel selection (sometimes called fitness proportionate selection) and tournament selection. The former gives every chromosome a chance of being picked, proportionate to its fitness. In tournament selection, a certain number of random chromosomes are challenged against one another, and the one with the best fitness is selected. 

In the __init__ method the initial_population is the chromosomes in the first generation of the algorithm. threshold is the fitness level that indicates that a solution has been found for the problem that the genetic algorithm is trying to solve. max_generations is the maximum number of generations to run. If we have run that many generations and no solution with a fitness level beyond threshold has been found, the best solution that has been found will be returned. mutation_chance is the probability of each chromosome in each generation mutating. crossover_chance is the probability that two parents selected to reproduce have children that are a mixture of their genes; otherwise, the children are just duplicates of the parents. Finally, selection_type is the type of selection method to use, as delineated by the enum SelectionType. 

In [2]:
from typing import TypeVar, Generic, List, Tuple, Callable
from enum import Enum
from random import choices, random
from heapq import nlargest
from statistics import mean

C = TypeVar('C', bound=Chromosome) # type of the chromosomes

class GeneticAlgorithm(Generic[C]):
    SelectionType = Enum("SelectionType", "ROULETTE TOURNAMENT")
    
    def __init__(self, initial_population: List[C], threshold: float, max_generations: int = 100, 
                 mutation_chance: float = 0.01, crossover_chance: float = 0.7, 
                 selection_type: SelectionType = SelectionType.TOURNAMENT) -> None:
        self._population: List[C] = initial_population
        self._threshold: float = threshold
        self._max_generations: int = max_generations
        self._mutation_chance: float = mutation_chance
        self._crossover_chance: float = crossover_chance
        self._selection_type: GeneticAlgorithm.SelectionType = selection_type
        self._fitness_key: Callable = type(self._population[0]).fitness
    
    
    # Use the probability distribution wheel to pick 2 parents
    # Note: will not work with negative fitness results
    def _pick_roulette(self, wheel: List[float]) -> Tuple[C, C]:
        return tuple(choices(self._population, weights=wheel, k=2))
    
    
    # randomly pick num_participants from _population. Then use
    # the nlargest() function from the heapq module to find 
    # the two largest individuals by _fitness_key
    def _pick_tournament(self, num_participants: int) -> Tuple[C, C]:
        participants: List[C] = choices(self._population, k=num_participants)
        return tuple(nlargest(2, participants, key=self._fitness_key))
    
    
    # Replace the population with a new generation of individuals
    def _reproduce_and_replace(self) -> None:
        new_population: List[C] = []
        # keep going until we've filled the new generation
        while len(new_population) < len(self._population):
            # pick the 2 parents
            if self._selection_type == GeneticAlgorithm.SelectionType.ROULETTE:
                parents: Tuple[C, C] = self._pick_roulette([x.fitness() for x in self._population])
            else:
                parents = self._pick_tournament(len(self._population) // 2)
                
            # potentially crossover the 2 parents
            if random() < self._crossover_chance:
                new_population.extend(parents[0].crossover(parents[1]))
            else:
                new_population.extend(parents)
                
        # if we had an odd number, we'll have 1 extra, so we remove it
        if len(new_population) > len(self._population):
            new_population.pop()
        self._population = new_population # replace reference
        
        
    # With _mutation_chance probability mutate each individual
    def _mutate(self) -> None:
        for individual in self._population:
            if random() < self._mutation_chance:
                individual.mutate()
                
                
    # Run the genetic algorithm for max_generations iterations
    # and return the best individual found
    def run(self) -> C:
        best: C = max(self._population, key=self._fitness_key)
        for generation in range(self._max_generations):
            # early exit if we beat threshold
            if best.fitness() >= self._threshold: 
                return best
            
            print(f"Generation {generation} Best {best.fitness()} Avg {mean(map(self._fitness_key, self._population))}")
            self._reproduce_and_replace()
            self._mutate()
            highest: C = max(self._population, key=self._fitness_key)
            if highest.fitness() > best.fitness():
                best = highest # found a new best
        return best # best we found in _max_generations

## Naive Test
As a test, we will start by implementing a simple problem that can be easily solved using traditional methods. We will try to maximize the equation 6x – x2 + 4y – y2. In other words, what values for x and y in that equation will yield the largest number?

In [3]:
from random import randrange, random
from copy import deepcopy

class SimpleEquation(Chromosome):
    def __init__(self, x: int, y: int) -> None:
        self.x: int = x
        self.y: int = y

    def fitness(self) -> float: # 6x - x^2 + 4y - y^2
        return 6 * self.x - self.x * self.x + 4 * self.y - self.y * self.y
    @classmethod
    def random_instance(cls) -> SimpleEquation:
        return SimpleEquation(randrange(100), randrange(100))

    def crossover(self, other: SimpleEquation) -> Tuple[SimpleEquation,
     SimpleEquation]:
        child1: SimpleEquation = deepcopy(self)
        child2: SimpleEquation = deepcopy(other)
        child1.y = other.y
        child2.y = self.y
        return child1, child2

    def mutate(self) -> None:
        if random() > 0.5: # mutate x
            if random() > 0.5:
                self.x += 1
            else:
                self.x -= 1
        else: # otherwise mutate y
            if random() > 0.5:
                self.y += 1
            else:
                self.y -= 1

    def __str__(self) -> str:
        return f"X: {self.x} Y: {self.y} Fitness: {self.fitness()}"

The method fitness() evaluates x and y using the equation 6x – x2 + 4y – y2. The higher the value, the more fit the individual chromosome is, according to Genetic-Algorithm. In the case of a random instance, x and y are initially set to be random integers between 0 and 100, so random_instance() does not need to do anything other than instantiate a new SimpleEquation with these values. To combine one Simple-Equation with another in crossover(), the y values of the two instances are simply swapped to create the two children. mutate() randomly increments or decrements x or y.

In [4]:
initial_population: List[SimpleEquation] = [SimpleEquation.random_instance() for _ in range(20)]
ga: GeneticAlgorithm[SimpleEquation] = GeneticAlgorithm(initial_population=initial_population, 
                                                        threshold=13.0, max_generations = 100,
                                                        mutation_chance = 0.1, crossover_chance = 0.7)
result: SimpleEquation = ga.run()
print(result)

Generation 0 Best -88 Avg -5078.85
Generation 1 Best -87 Avg -1072.85
Generation 2 Best -87 Avg -133.55
Generation 3 Best -87 Avg -87.05
Generation 4 Best -68 Avg -87.2
Generation 5 Best -68 Avg -84.15
Generation 6 Best -68 Avg -74.7
Generation 7 Best -51 Avg -66.45
Generation 8 Best -36 Avg -56.3
Generation 9 Best -36 Avg -45.75
Generation 10 Best -36 Avg -38.4
Generation 11 Best -36 Avg -36
Generation 12 Best -23 Avg -34.2
Generation 13 Best -23 Avg -27.55
Generation 14 Best -23 Avg -23.8
Generation 15 Best -23 Avg -23
Generation 16 Best -23 Avg -23.05
Generation 17 Best -23 Avg -23.7
Generation 18 Best -12 Avg -21.95
Generation 19 Best -12 Avg -19.9
Generation 20 Best -3 Avg -11.8
Generation 21 Best -3 Avg -8.95
Generation 22 Best 4 Avg -4.1
Generation 23 Best 9 Avg -1.4
Generation 24 Best 9 Avg 1.5
Generation 25 Best 12 Avg 8.6
Generation 26 Best 12 Avg 9.3
X: 3 Y: 2 Fitness: 13


## SEND+MORE=MONEY
A chromosome that represents the SEND+MORE=MONEY problem is represented in SendMoreMoney2.

In [5]:
from random import shuffle, sample

class SendMoreMoney2(Chromosome):
    def __init__(self, letters: List[str]) -> None:
        self.letters: List[str] = letters

    def fitness(self) -> float:
        s: int = self.letters.index("S")
        e: int = self.letters.index("E")
        n: int = self.letters.index("N")
        d: int = self.letters.index("D")
        m: int = self.letters.index("M")
        o: int = self.letters.index("O")
        r: int = self.letters.index("R")
        y: int = self.letters.index("Y")
        send: int = s * 1000 + e * 100 + n * 10 + d
        more: int = m * 1000 + o * 100 + r * 10 + e
        money: int = m * 10000 + o * 1000 + n * 100 + e * 10 + y
        difference: int = abs(money - (send + more))
        #Dividing 1 by a fitness value is a simple way to convert a minimization problem into a maximization problem.
        return 1 / (difference + 1) 

    @classmethod
    def random_instance(cls) -> SendMoreMoney2:
        letters = ["S", "E", "N", "D", "M", "O", "R", "Y", " ", " "]
        shuffle(letters)
        return SendMoreMoney2(letters)

    def crossover(self, other: SendMoreMoney2) -> Tuple[SendMoreMoney2,
     SendMoreMoney2]:
        child1: SendMoreMoney2 = deepcopy(self)
        child2: SendMoreMoney2 = deepcopy(other)
            
        idx1, idx2 = sample(range(len(self.letters)), k=2)
        
        l1, l2 = child1.letters[idx1], child2.letters[idx2]
        
        child1.letters[child1.letters.index(l2)], child1.letters[idx2] = child1.letters[idx2], l2
        child2.letters[child2.letters.index(l1)], child2.letters[idx1] = child2.letters[idx1], l1
        
        return child1, child2

    def mutate(self) -> None: # swap two letters' locations
        idx1, idx2 = sample(range(len(self.letters)), k=2)
        self.letters[idx1], self.letters[idx2] = self.letters[idx2], self.letters[idx1]

    def __str__(self) -> str:
        s: int = self.letters.index("S")
        e: int = self.letters.index("E")
        n: int = self.letters.index("N")
        d: int = self.letters.index("D")
        m: int = self.letters.index("M")
        o: int = self.letters.index("O")
        r: int = self.letters.index("R")
        y: int = self.letters.index("Y")
        send: int = s * 1000 + e * 100 + n * 10 + d
        more: int = m * 1000 + o * 100 + r * 10 + e
        money: int = m * 10000 + o * 1000 + n * 100 + e * 10 + y
        difference: int = abs(money - (send + more))
        return f"{send} + {more} = {money} Difference: {difference}"

We can plug SendMoreMoney2 into GeneticAlgorithm just as easily as we plugged in SimpleEquation. This is a fairly tough problem, and it will take a long time to execute if the parameters are not well tweaked. And there’s still some randomness even if one gets them right! The problem may be solved in a few seconds or a few minutes. Unfortunately, that is the nature of genetic algorithms. 

In [6]:
initial_population: List[SendMoreMoney2] = [SendMoreMoney2.random_instance() for _ in range(1000)]
ga: GeneticAlgorithm[SendMoreMoney2] = GeneticAlgorithm(initial_population=initial_population, 
                                                        threshold=1.0, max_generations = 1000,
                                                        mutation_chance = 0.2, crossover_chance = 0.7, 
                                                        selection_type=GeneticAlgorithm.SelectionType.ROULETTE)
result: SendMoreMoney2 = ga.run()
print(result)

Generation 0 Best 0.025 Avg 0.0001457122557245635
Generation 1 Best 0.5 Avg 0.005598252928113855
Generation 2 Best 0.5 Avg 0.12251012982351211
3829 + 458 = 4287 Difference: 0


This solution indicates that SEND = 8324, MORE = 913, and MONEY = 9237. How is that possible? It looks like letters are missing from the solution. In fact, if M = 0, there are several solutions to the problem not possible in the version from chapter 3. MORE is actually 0913 here, and MONEY is 09237. The 0 is just ignored. 

## List Compression
Suppose that we have some information we want to compress. Suppose that it is a list of items, and we do not care about the order of the items, as long as all of them are intact. What order of the items will maximize the compression ratio? Did you even know that the order of the items will affect the compression ratio for most compression algorithms?

The answer will depend on the compression algorithm used. For this example, we will use the compress() function from the zlib module with its standard settings. The solution is shown here in its entirety for a list of 12 first names. If we do not run the genetic algorithm and we just run compress() on the 12 names in the order they were originally presented, the resulting compressed data will be 165 bytes. 

In [7]:
from zlib import compress
from sys import getsizeof
from pickle import dumps

# 165 bytes compressed
PEOPLE: List[str] = ["Michael", "Sarah", "Joshua", "Narine", "David",
     "Sajid", "Melanie", "Daniel", "Wei", "Dean", "Brian", "Murat", "Lisa"] 


class ListCompression(Chromosome):
    def __init__(self, lst: List[Any]) -> None:
        self.lst: List[Any] = lst

    @property
    def bytes_compressed(self) -> int:
        return getsizeof(compress(dumps(self.lst)))

    def fitness(self) -> float:
        return 1 / self.bytes_compressed

    @classmethod
    def random_instance(cls) -> ListCompression:
        mylst: List[str] = deepcopy(PEOPLE)
        shuffle(mylst)
        return ListCompression(mylst)

    def crossover(self, other: ListCompression) -> Tuple[ListCompression,
     ListCompression]:
        child1: ListCompression = deepcopy(self)
        child2: ListCompression = deepcopy(other)
            
        idx1, idx2 = sample(range(len(self.lst)), k=2)
        
        l1, l2 = child1.lst[idx1], child2.lst[idx2]
        
        child1.lst[child1.lst.index(l2)], child1.lst[idx2] = child1.lst[idx2], l2
        child2.lst[child2.lst.index(l1)], child2.lst[idx1] = child2.lst[idx1], l1
        
        return child1, child2

    def mutate(self) -> None: # swap two locations
        idx1, idx2 = sample(range(len(self.lst)), k=2)
        self.lst[idx1], self.lst[idx2] = self.lst[idx2], self.lst[idx1]

    def __str__(self) -> str:
        return f"Order: {self.lst} Bytes: {self.bytes_compressed}"

Note how similar this implementation is to the implementation from SEND+MORE=MONEY. The crossover() and mutate() functions are essentially the same. In both problems’ solutions, we are taking a list of items and continually rearranging them and testing those rearrangements. One could write a generic superclass for both problems’ solutions that would work with a wide variety of problems. Any problem that can be represented as a list of items that needs to find its optimal order could be solved the same way. The only real point of customization for the subclasses would be their respective fitness functions. 

In [8]:
initial_population: List[ListCompression] = [ListCompression.random_instance() for _ in range(1000)]
ga: GeneticAlgorithm[ListCompression] = GeneticAlgorithm(initial_population=initial_population, 
                                                         threshold=1.0, max_generations = 1000, 
                                                         mutation_chance = 0.2, crossover_chance = 0.7, 
                                                         selection_type=GeneticAlgorithm.SelectionType.TOURNAMENT)
result: ListCompression = ga.run()
print(result)

Generation 0 Best 0.006172839506172839 Avg 0.006049952017048166
Generation 1 Best 0.006211180124223602 Avg 0.006090818191233948
Generation 2 Best 0.006211180124223602 Avg 0.006123881628496068
Generation 3 Best 0.006211180124223602 Avg 0.006162512200749813
Generation 4 Best 0.006211180124223602 Avg 0.00616795604188583
Generation 5 Best 0.006211180124223602 Avg 0.006166685047684451
Generation 6 Best 0.006211180124223602 Avg 0.0061719440336855105
Generation 7 Best 0.006211180124223602 Avg 0.006171864111434478
Generation 8 Best 0.006211180124223602 Avg 0.006178250554047254
Generation 9 Best 0.006211180124223602 Avg 0.006185951075260012
Generation 10 Best 0.006211180124223602 Avg 0.0061900232746894554
Generation 11 Best 0.006211180124223602 Avg 0.00619053424942791
Generation 12 Best 0.006211180124223602 Avg 0.006193754406476318
Generation 13 Best 0.006211180124223602 Avg 0.00619385100908944
Generation 14 Best 0.006211180124223602 Avg 0.006193780787379312
Generation 15 Best 0.006211180124223

Generation 135 Best 0.00625 Avg 0.006227214130481018
Generation 136 Best 0.00625 Avg 0.006228393949942786
Generation 137 Best 0.00625 Avg 0.006225977279252109
Generation 138 Best 0.00625 Avg 0.006225261388609555
Generation 139 Best 0.00625 Avg 0.006226421455739897
Generation 140 Best 0.00625 Avg 0.006227557308004638
Generation 141 Best 0.00625 Avg 0.006224551343872178
Generation 142 Best 0.00625 Avg 0.006228632575117505
Generation 143 Best 0.00625 Avg 0.006229153356871291
Generation 144 Best 0.00625 Avg 0.006224533848281062
Generation 145 Best 0.00625 Avg 0.006226710646226176
Generation 146 Best 0.00625 Avg 0.0062254151451184075
Generation 147 Best 0.00625 Avg 0.006227398504539143
Generation 148 Best 0.00625 Avg 0.006226459438875352
Generation 149 Best 0.00625 Avg 0.006226379070328243
Generation 150 Best 0.00625 Avg 0.006225373460818777
Generation 151 Best 0.00625 Avg 0.006224978958706785
Generation 152 Best 0.00625 Avg 0.006225663608992077
Generation 153 Best 0.00625 Avg 0.00622622155

Generation 290 Best 0.00625 Avg 0.006223265088682602
Generation 291 Best 0.00625 Avg 0.006226720517665352
Generation 292 Best 0.00625 Avg 0.006226061872285947
Generation 293 Best 0.00625 Avg 0.006224972979696774
Generation 294 Best 0.00625 Avg 0.006228381914408514
Generation 295 Best 0.00625 Avg 0.00622627473230378
Generation 296 Best 0.00625 Avg 0.006225614813745955
Generation 297 Best 0.00625 Avg 0.006225964898476998
Generation 298 Best 0.00625 Avg 0.006228338364332719
Generation 299 Best 0.00625 Avg 0.006226246263763803
Generation 300 Best 0.00625 Avg 0.0062264131589478645
Generation 301 Best 0.00625 Avg 0.006226465153837765
Generation 302 Best 0.00625 Avg 0.006224816725106003
Generation 303 Best 0.00625 Avg 0.006227526322267508
Generation 304 Best 0.00625 Avg 0.0062286517547818126
Generation 305 Best 0.00625 Avg 0.006225791222873559
Generation 306 Best 0.00625 Avg 0.0062231631051518545
Generation 307 Best 0.00625 Avg 0.006229592828767108
Generation 308 Best 0.00625 Avg 0.0062255676

Generation 445 Best 0.00625 Avg 0.00622258096008256
Generation 446 Best 0.00625 Avg 0.006225480655502537
Generation 447 Best 0.00625 Avg 0.006229340946727697
Generation 448 Best 0.00625 Avg 0.006228055394313524
Generation 449 Best 0.00625 Avg 0.006226200027718129
Generation 450 Best 0.00625 Avg 0.006227291334015662
Generation 451 Best 0.00625 Avg 0.006225662787141973
Generation 452 Best 0.00625 Avg 0.006224977580370143
Generation 453 Best 0.00625 Avg 0.006227277733508976
Generation 454 Best 0.00625 Avg 0.006222428641770643
Generation 455 Best 0.00625 Avg 0.006227199692177873
Generation 456 Best 0.00625 Avg 0.006225718783360903
Generation 457 Best 0.00625 Avg 0.006223991279393414
Generation 458 Best 0.00625 Avg 0.006224345702962129
Generation 459 Best 0.00625 Avg 0.00622367651208129
Generation 460 Best 0.00625 Avg 0.0062257933957765455
Generation 461 Best 0.00625 Avg 0.006222601046615947
Generation 462 Best 0.00625 Avg 0.006227405536131983
Generation 463 Best 0.00625 Avg 0.0062271920249

Generation 600 Best 0.00625 Avg 0.006226573826760841
Generation 601 Best 0.00625 Avg 0.0062258207179710775
Generation 602 Best 0.00625 Avg 0.0062256632991078
Generation 603 Best 0.00625 Avg 0.006225664035971053
Generation 604 Best 0.00625 Avg 0.006227328666173443
Generation 605 Best 0.00625 Avg 0.006226235955683957
Generation 606 Best 0.00625 Avg 0.006227338527729269
Generation 607 Best 0.00625 Avg 0.0062256185279234814
Generation 608 Best 0.00625 Avg 0.00622591096634778
Generation 609 Best 0.00625 Avg 0.006227312283529796
Generation 610 Best 0.00625 Avg 0.0062249593348845155
Generation 611 Best 0.00625 Avg 0.006226465007510726
Generation 612 Best 0.00625 Avg 0.006226863958544941
Generation 613 Best 0.00625 Avg 0.006226279428929092
Generation 614 Best 0.00625 Avg 0.006226487489054946
Generation 615 Best 0.00625 Avg 0.0062255981072200225
Generation 616 Best 0.00625 Avg 0.006224412179202508
Generation 617 Best 0.00625 Avg 0.006227809951869932
Generation 618 Best 0.00625 Avg 0.00622369149

Generation 755 Best 0.00625 Avg 0.006221499738896706
Generation 756 Best 0.00625 Avg 0.006224905632476863
Generation 757 Best 0.00625 Avg 0.006224614149640116
Generation 758 Best 0.00625 Avg 0.0062270799902771415
Generation 759 Best 0.00625 Avg 0.006227384765469976
Generation 760 Best 0.00625 Avg 0.006227074258533895
Generation 761 Best 0.00625 Avg 0.006226423964135443
Generation 762 Best 0.00625 Avg 0.006223518195638079
Generation 763 Best 0.00625 Avg 0.0062239052729681
Generation 764 Best 0.00625 Avg 0.0062290223214023464
Generation 765 Best 0.00625 Avg 0.006223524688770882
Generation 766 Best 0.00625 Avg 0.006226771348093213
Generation 767 Best 0.00625 Avg 0.006223421213753243
Generation 768 Best 0.00625 Avg 0.006225326727267218
Generation 769 Best 0.00625 Avg 0.006228791183601505
Generation 770 Best 0.00625 Avg 0.006225091704808288
Generation 771 Best 0.00625 Avg 0.006227030358916556
Generation 772 Best 0.00625 Avg 0.006226456572788589
Generation 773 Best 0.00625 Avg 0.006227890722

Generation 910 Best 0.00625 Avg 0.006228305523916318
Generation 911 Best 0.00625 Avg 0.006228149980763768
Generation 912 Best 0.00625 Avg 0.0062264041945994265
Generation 913 Best 0.00625 Avg 0.006224957085561825
Generation 914 Best 0.00625 Avg 0.006226045749511459
Generation 915 Best 0.00625 Avg 0.006228047062460372
Generation 916 Best 0.00625 Avg 0.006226208803559268
Generation 917 Best 0.00625 Avg 0.006224800736068728
Generation 918 Best 0.00625 Avg 0.006226513707431235
Generation 919 Best 0.00625 Avg 0.006226962087192696
Generation 920 Best 0.00625 Avg 0.0062276270682940105
Generation 921 Best 0.00625 Avg 0.006227380309011373
Generation 922 Best 0.00625 Avg 0.006226281250646949
Generation 923 Best 0.00625 Avg 0.00622441864817527
Generation 924 Best 0.00625 Avg 0.0062231925024065805
Generation 925 Best 0.00625 Avg 0.006230325273773256
Generation 926 Best 0.00625 Avg 0.006225921675576895
Generation 927 Best 0.00625 Avg 0.006225315196813857
Generation 928 Best 0.00625 Avg 0.0062227561

## Summary
Genetic algorithms are not a panacea. In fact, they are not suitable for most problems. For any problem in which a fast deterministic algorithm exists, a genetic algorithm approach does not make sense. Their inherently stochastic nature makes their runtimes unpredictable. To solve this problem, they can be cut off after a certain number of generations. But then it is not clear if a truly optimal solution has been found. 

Genetic algorithms have been shown to find suboptimal, but pretty good, solutions in short periods of time. The problem is widely applicable to the efficient distribution of goods. For example, dispatchers of FedEx and UPS trucks use software to solve the Traveling Salesman problem every day. Algorithms that help solve the problem can cut costs in a large variety of industries. 